In [104]:
import pandas as pd

In [247]:
etrade_data = "/Users/sanathnair/Developer/trading/data/etrade/etrade1_data.html"

In [248]:
dfs = pd.read_html(etrade_data)

In [250]:
dfs[0].columns

Index(['Transaction Status', 'Order Description', 'Date & Time', 'Quantity',
       'Price Executed', 'Commission/Fee'],
      dtype='object')

In [88]:
df = dfs[0]

In [89]:
df["Date & Time"] = pd.to_datetime(df["Date & Time"], format = "%m/%d/%y %I:%M:%S %p EDT")

In [91]:
df["Date & Time"] = df["Date & Time"] - pd.Timedelta(hours = 3)

In [92]:
df

,Transaction Status,Order Description,Date & Time,Quantity,Price Executed,Commission/Fee
0,Order Placed,Sell 6 AGL Mkt Day,2023-09-01 09:41:14,6,â,0.0017
1,Executed,Sell 1 AGL Mkt Day,2023-09-01 09:41:15,1,17.93,0.0101
2,Executed,Sell 5 AGL Mkt Day,2023-09-01 09:41:15,5,17.9301,0.0107


In [57]:
df[["Action", "Symbol"]] = df["Order Description"].str.split(expand = True)[[0, 2]]

In [58]:
df

,Transaction Status,Order Description,Date & Time,Quantity,Price Executed,Commission/Fee,Action,Symbol
0,Order Placed,Sell 6 AGL Mkt Day,2023-09-01 09:41:14,6,â,0.0017,Sell,AGL
1,Executed,Sell 1 AGL Mkt Day,2023-09-01 09:41:15,1,17.93,0.0101,Sell,AGL
2,Executed,Sell 5 AGL Mkt Day,2023-09-01 09:41:15,5,17.9301,0.0107,Sell,AGL


In [61]:
df = df.drop(["Order Description", "Commission/Fee", "Transaction Status"], axis = 1)
df = df.drop([0])

In [62]:
df

,Date & Time,Quantity,Price Executed,Action,Symbol
1,2023-09-01 09:41:15,1,17.93,Sell,AGL
2,2023-09-01 09:41:15,5,17.9301,Sell,AGL


In [63]:
df.dtypes

Date & Time       datetime64[ns]
Quantity                   int64
Price Executed            object
Action                    object
Symbol                    object
dtype: object

In [64]:
df["Price Executed"] = pd.to_numeric(df["Price Executed"])

In [66]:
df["Dollar Amt"] = df["Quantity"] * df["Price Executed"]

In [69]:
df["Broker Executed"] = df["Date & Time"].dt.strftime('%I:%M:%S')

In [70]:
df

,Date & Time,Quantity,Price Executed,Action,Symbol,Dollar Amt,Broker Executed
1,2023-09-01 09:41:15,1,17.9300,Sell,AGL,17.9300,09:41:15
2,2023-09-01 09:41:15,5,17.9301,Sell,AGL,89.6505,09:41:15


In [72]:
df = df.rename(columns = {
    "Quantity": "Size",
    "Price Executed": "Price",
})

In [73]:
df

,Date & Time,Size,Price,Action,Symbol,Dollar Amt,Broker Executed
1,2023-09-01 09:41:15,1,17.9300,Sell,AGL,17.9300,09:41:15
2,2023-09-01 09:41:15,5,17.9301,Sell,AGL,89.6505,09:41:15


In [74]:
def parse_etrade_report(df):
    df["Date & Time"] = pd.to_datetime(df["Date & Time"], format = "%m/%d/%y %I:%M:%S %p EDT")
    df["Date & Time"] = df["Date & Time"] - pd.Timedelta(hours = 3)

    df[["Action", "Symbol"]] = df["Order Description"].str.split(expand = True)[[0, 2]]

    df = df.drop(["Order Description", "Commission/Fee", "Transaction Status"], axis = 1)
    df = df.drop([0])

    df["Price Executed"] = pd.to_numeric(df["Price Executed"])
    df["Dollar Amt"] = df["Quantity"] * df["Price Executed"]

    df["Broker Executed"] = df["Date & Time"].dt.strftime('%I:%M:%S')

    df = df.rename(columns = {
        "Quantity": "Size",
        "Price Executed": "Price",
    })
    return df

In [99]:
etrade2_data = "/Users/sanathnair/Developer/trading/data/etrade/etrade1_data.html"
dfs = pd.read_html(etrade2_data)
master_df = pd.DataFrame()
for df in dfs:
    parsed_df = parse_etrade_report(df)
    master_df = pd.concat([master_df, parsed_df], ignore_index = True)
master_df.to_csv("all_etrade.csv", index = False)

In [100]:
etrade2_data = "/Users/sanathnair/Developer/trading/data/etrade/etrade2_data.html"
dfs = pd.read_html(etrade2_data)
master_df = pd.DataFrame()
for df in dfs:
    parsed_df = parse_etrade_report(df)
    master_df = pd.concat([master_df, parsed_df], ignore_index = True)
master_df.to_csv("all_etrade2.csv", index = False)

In [141]:
df = (pd.read_csv("/Users/sanathnair/Developer/trading/data/etrade/all_etrade2.csv"))
df["Broker"] = "E2"

In [118]:
df["Date & Time"] = pd.to_datetime(df["Date & Time"])

In [119]:
df.dtypes

Date & Time        datetime64[ns]
Size                        int64
Price                     float64
Action                     object
Symbol                     object
Dollar Amt                float64
Broker Executed            object
dtype: object

In [120]:
df["Date"] = df["Date & Time"].dt.strftime("%m/%d/%y")

In [121]:
df = df.drop(columns = ["Date & Time"])

In [142]:
df

,Size,Price,Action,Symbol,Dollar Amt,Broker Executed,Date,Broker
0,8,10.5418,Sell,BYN,84.3344,09:33:46,09/01/23,E2
1,1,10.5418,Sell,BYN,10.5418,09:33:46,09/01/23,E2
2,1,31.4000,Buy,GBCI,31.4000,08:35:15,09/01/23,E2
3,2,31.4000,Buy,GBCI,62.8000,08:35:15,09/01/23,E2
4,8,10.6871,Buy,SMAP,85.4968,07:55:43,09/01/23,E2
...,...,...,...,...,...,...,...,...
200,7,10.3792,Buy,CDNA,72.6544,11:37:07,07/18/23,E2
201,3,5.1750,Sell,ICCC,15.5250,11:24:25,07/18/23,E2
202,17,5.0800,Sell,ICCC,86.3600,11:24:25,07/18/23,E2
203,77,1.2950,Buy,EDUC,99.7150,07:12:44,07/18/23,E2


In [143]:
df.to_csv("/Users/sanathnair/Developer/trading/data/etrade/all_etrade2.csv", index = False)

In [221]:
report_df = pd.read_csv("/Users/sanathnair/Developer/trading/reports/original/report_08_25.csv")
df = pd.read_csv("/Users/sanathnair/Developer/trading/data/etrade/all_etrade.csv")

In [222]:
report_df

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
0,08/25/23,06:44:05:103673,06:44:10:565472,NaN,NRG,FD,Buy,3.0,NaN,NaN,...,37.290,37.31,37.290,37.31,51017.0,51020.0,Market,False,NaN,NaN
1,08/25/23,06:44:12:161752,06:44:22:606364,NaN,NRG,IF,Buy,3.0,NaN,NaN,...,37.290,37.31,37.290,37.31,51020.0,51225.0,Market,False,NaN,NaN
2,08/25/23,06:44:25:166560,06:44:32:264227,NaN,NRG,SB,Buy,3.0,NaN,NaN,...,37.290,37.31,37.270,37.29,51225.0,51855.0,Market,False,NaN,NaN
3,08/25/23,06:44:34:905819,06:44:35:829129,1692971075753,NRG,ET,Buy,3.0,NaN,NaN,...,37.260,37.28,37.260,37.28,51857.0,51857.0,Market,False,43974,NaN
4,08/25/23,06:44:36:761912,06:44:37:719205,1692971077639,NRG,E2,Buy,3.0,NaN,NaN,...,36.900,37.69,36.900,37.69,46.0,46.0,Market,False,4516,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,08/25/23,11:07:19:205959,11:07:25:111640,NaN,SNSE,FD,Sell,101.0,NaN,NaN,...,0.961,1.02,0.961,1.02,10487.0,10588.0,Market,False,NaN,NaN
1285,08/25/23,11:07:28:369274,11:07:35:857648,NaN,SNSE,SB,Sell,101.0,NaN,NaN,...,0.961,1.02,0.961,1.02,10588.0,10689.0,Market,False,NaN,NaN
1286,08/25/23,11:10:47:400107,11:10:54:354142,NaN,ADRT,SB,Sell,9.0,NaN,NaN,...,11.400,11.51,11.400,11.51,47111.0,47120.0,Market,False,NaN,NaN
1287,08/25/23,11:13:04:867837,11:13:14:841947,NaN,PFMT,IF,Sell,43.0,NaN,NaN,...,2.310,2.33,2.310,2.33,70625.0,70668.0,Market,False,NaN,NaN


In [183]:
copy = report_df[report_df["Broker"] == "ET"].copy()

In [138]:
df[df["Date"] == "08/25/23"]

,Size,Price,Action,Symbol,Dollar Amt,Broker Executed,Date,Broker
45,10,4.9350,Sell,INZY,49.3500,10:39:07,08/25/23,ET
46,10,4.9350,Sell,INZY,49.3500,10:39:07,08/25/23,ET
47,77,1.3279,Buy,MICS,102.2483,09:43:08,08/25/23,ET
48,1,1.3250,Buy,MICS,1.3250,09:43:08,08/25/23,ET
49,1,8.5975,Buy,CDNA,8.5975,07:34:26,08/25/23,ET
50,11,8.5975,Buy,CDNA,94.5725,07:34:26,08/25/23,ET


In [154]:
merged = pd.merge(left = report_df, right = df, on = ["Date", "Symbol", "Action", "Broker"])
merged

,Date,Program Submitted,Program Executed,Broker Executed_x,Symbol,Broker,Action,Size_x,Price_x,Dollar Amt_x,...,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID,Size_y,Price_y,Dollar Amt_y,Broker Executed_y
0,08/25/23,07:34:25:077046,07:34:26:145261,1692974065974,CDNA,ET,Buy,12.0,8.5975,103.17,...,48055.0,48068.0,Market,False,44014,NaN,1,8.5975,8.5975,07:34:26
1,08/25/23,07:34:25:077046,07:34:26:145261,1692974065974,CDNA,ET,Buy,12.0,8.5975,103.17,...,48055.0,48068.0,Market,False,44014,NaN,11,8.5975,94.5725,07:34:26
2,08/25/23,09:43:07:934369,09:43:08:655465,1692981788644,MICS,ET,Buy,78.0,NaN,NaN,...,2520.0,2520.0,Market,False,44104,NaN,77,1.3279,102.2483,09:43:08
3,08/25/23,09:43:07:934369,09:43:08:655465,1692981788644,MICS,ET,Buy,78.0,NaN,NaN,...,2520.0,2520.0,Market,False,44104,NaN,1,1.3250,1.3250,09:43:08
4,08/25/23,10:39:06:484051,10:39:07:199139,1692985147199,INZY,ET,Sell,20.0,NaN,NaN,...,175891.0,175891.0,Market,False,44148,NaN,10,4.9350,49.3500,10:39:07
5,08/25/23,10:39:06:484051,10:39:07:199139,1692985147199,INZY,ET,Sell,20.0,NaN,NaN,...,175891.0,175891.0,Market,False,44148,NaN,10,4.9350,49.3500,10:39:07


In [147]:
merged["Split"] = True

In [155]:
merged = merged.drop(columns = ["Size_x", "Price_x", "Dollar Amt_x", "Broker Executed_x"])

In [161]:
merged = merged.rename(columns = {
    "Size_y": "Size",
    "Price_y": "Price",
    "Dollar Amt_y": "Dollar Amt",
    "Broker Executed_y": "Broker Executed"
})
merged

,Date,Program Submitted,Program Executed,Symbol,Broker,Action,Pre Quote,Post Quote,Pre Bid,Pre Ask,...,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID,Size,Price,Dollar Amt,Broker Executed
0,08/25/23,07:34:25:077046,07:34:26:145261,CDNA,ET,Buy,8.590,8.590,8.59,8.60,...,48055.0,48068.0,Market,False,44014,NaN,1,8.5975,8.5975,07:34:26
1,08/25/23,07:34:25:077046,07:34:26:145261,CDNA,ET,Buy,8.590,8.590,8.59,8.60,...,48055.0,48068.0,Market,False,44014,NaN,11,8.5975,94.5725,07:34:26
2,08/25/23,09:43:07:934369,09:43:08:655465,MICS,ET,Buy,1.290,1.290,1.29,1.36,...,2520.0,2520.0,Market,False,44104,NaN,77,1.3279,102.2483,09:43:08
3,08/25/23,09:43:07:934369,09:43:08:655465,MICS,ET,Buy,1.290,1.290,1.29,1.36,...,2520.0,2520.0,Market,False,44104,NaN,1,1.3250,1.3250,09:43:08
4,08/25/23,10:39:06:484051,10:39:07:199139,INZY,ET,Sell,4.935,4.935,4.93,4.94,...,175891.0,175891.0,Market,False,44148,NaN,10,4.9350,49.3500,10:39:07
5,08/25/23,10:39:06:484051,10:39:07:199139,INZY,ET,Sell,4.935,4.935,4.93,4.94,...,175891.0,175891.0,Market,False,44148,NaN,10,4.9350,49.3500,10:39:07


In [163]:
column_order = ['Date', 'Program Submitted', 'Program Executed', 'Broker Executed', 'Symbol', 'Broker', 'Action', 'Size', 'Price', 'Dollar Amt', 'Pre Quote', 'Post Quote', 'Pre Bid', 'Pre Ask', 'Post Bid', 'Post Ask', 'Pre Volume', 'Post Volume', 'Order Type', 'Split', 'Order ID', 'Activity ID']
merged = merged.reindex(columns = column_order)


In [164]:
merged

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
0,08/25/23,07:34:25:077046,07:34:26:145261,07:34:26,CDNA,ET,Buy,1,8.5975,8.5975,...,8.59,8.60,8.59,8.60,48055.0,48068.0,Market,False,44014,NaN
1,08/25/23,07:34:25:077046,07:34:26:145261,07:34:26,CDNA,ET,Buy,11,8.5975,94.5725,...,8.59,8.60,8.59,8.60,48055.0,48068.0,Market,False,44014,NaN
2,08/25/23,09:43:07:934369,09:43:08:655465,09:43:08,MICS,ET,Buy,77,1.3279,102.2483,...,1.29,1.36,1.29,1.36,2520.0,2520.0,Market,False,44104,NaN
3,08/25/23,09:43:07:934369,09:43:08:655465,09:43:08,MICS,ET,Buy,1,1.3250,1.3250,...,1.29,1.36,1.29,1.36,2520.0,2520.0,Market,False,44104,NaN
4,08/25/23,10:39:06:484051,10:39:07:199139,10:39:07,INZY,ET,Sell,10,4.9350,49.3500,...,4.93,4.94,4.93,4.94,175891.0,175891.0,Market,False,44148,NaN
5,08/25/23,10:39:06:484051,10:39:07:199139,10:39:07,INZY,ET,Sell,10,4.9350,49.3500,...,4.93,4.94,4.93,4.94,175891.0,175891.0,Market,False,44148,NaN


In [180]:
copy

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
3,08/25/23,06:44:34:905819,06:44:35:829129,1692971075753,NRG,ET,Buy,3.0,NaN,NaN,...,37.26,37.28,37.26,37.28,51857.0,51857.0,Market,False,43974,NaN
10,08/25/23,06:45:13:696128,06:45:14:387174,1692971114310,FRBA,ET,Buy,9.0,11.7300,105.5700,...,11.63,11.83,11.63,11.83,856.0,856.0,Market,False,43975,NaN
17,08/25/23,06:45:48:667177,06:45:49:362800,1692971149291,GIS,ET,Buy,1.0,NaN,NaN,...,68.33,68.35,68.33,68.34,111801.0,112378.0,Market,False,43976,NaN
24,08/25/23,06:46:24:323590,06:46:25:008668,1692971184936,SCKT,ET,Buy,77.0,1.3183,101.5091,...,1.30,1.32,1.30,1.32,280.0,357.0,Market,False,43977,NaN
31,08/25/23,06:50:31:797780,06:50:32:864647,1692971432787,NRG,ET,Sell,3.0,NaN,NaN,...,37.27,37.28,37.27,37.28,68718.0,68721.0,Market,False,43978,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258,08/25/23,10:59:38:969726,10:59:39:991388,1692986379916,OXM,ET,Sell,1.0,NaN,NaN,...,101.22,101.41,101.22,101.41,109744.0,109744.0,Market,False,44165,NaN
1265,08/25/23,11:00:04:588376,11:00:05:369784,1692986405363,APT,ET,Sell,22.0,NaN,NaN,...,4.49,4.54,4.49,4.54,26002.0,26002.0,Market,False,44166,NaN
1271,08/25/23,11:00:29:894407,11:00:30:792877,1692986430773,ASB,ET,Sell,6.0,NaN,NaN,...,16.91,16.92,16.91,16.92,655796.0,655802.0,Market,False,44167,NaN
1277,08/25/23,11:03:06:100609,11:03:06:813391,1692986586818,SNSE,ET,Buy,101.0,NaN,NaN,...,0.98,1.02,0.98,1.02,7682.0,7783.0,Market,False,44168,NaN


In [189]:
copy = copy.drop(copy[copy["Order ID"].isin(merged["Order ID"])].index)

In [190]:
pd.concat([copy, merged], ignore_index = True)

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
0,08/25/23,06:44:34:905819,06:44:35:829129,1692971075753,NRG,ET,Buy,3.0,NaN,NaN,...,37.26,37.28,37.26,37.28,51857.0,51857.0,Market,False,43974,NaN
1,08/25/23,06:45:13:696128,06:45:14:387174,1692971114310,FRBA,ET,Buy,9.0,11.7300,105.5700,...,11.63,11.83,11.63,11.83,856.0,856.0,Market,False,43975,NaN
2,08/25/23,06:45:48:667177,06:45:49:362800,1692971149291,GIS,ET,Buy,1.0,NaN,NaN,...,68.33,68.35,68.33,68.34,111801.0,112378.0,Market,False,43976,NaN
3,08/25/23,06:46:24:323590,06:46:25:008668,1692971184936,SCKT,ET,Buy,77.0,1.3183,101.5091,...,1.30,1.32,1.30,1.32,280.0,357.0,Market,False,43977,NaN
4,08/25/23,06:50:31:797780,06:50:32:864647,1692971432787,NRG,ET,Sell,3.0,NaN,NaN,...,37.27,37.28,37.27,37.28,68718.0,68721.0,Market,False,43978,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,08/25/23,07:34:25:077046,07:34:26:145261,07:34:26,CDNA,ET,Buy,11.0,8.5975,94.5725,...,8.59,8.60,8.59,8.60,48055.0,48068.0,Market,False,44014,NaN
195,08/25/23,09:43:07:934369,09:43:08:655465,09:43:08,MICS,ET,Buy,77.0,1.3279,102.2483,...,1.29,1.36,1.29,1.36,2520.0,2520.0,Market,False,44104,NaN
196,08/25/23,09:43:07:934369,09:43:08:655465,09:43:08,MICS,ET,Buy,1.0,1.3250,1.3250,...,1.29,1.36,1.29,1.36,2520.0,2520.0,Market,False,44104,NaN
197,08/25/23,10:39:06:484051,10:39:07:199139,10:39:07,INZY,ET,Sell,10.0,4.9350,49.3500,...,4.93,4.94,4.93,4.94,175891.0,175891.0,Market,False,44148,NaN


In [225]:
et = report_df[report_df["Broker"] == "ET"]
report_df = report_df.drop(et[et["Order ID"].isin(merged["Order ID"])].index)
pd.concat([report_df, merged], ignore_index = True)

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
0,08/25/23,06:44:05:103673,06:44:10:565472,NaN,NRG,FD,Buy,3.0,NaN,NaN,...,37.29,37.31,37.29,37.31,51017.0,51020.0,Market,False,NaN,NaN
1,08/25/23,06:44:12:161752,06:44:22:606364,NaN,NRG,IF,Buy,3.0,NaN,NaN,...,37.29,37.31,37.29,37.31,51020.0,51225.0,Market,False,NaN,NaN
2,08/25/23,06:44:25:166560,06:44:32:264227,NaN,NRG,SB,Buy,3.0,NaN,NaN,...,37.29,37.31,37.27,37.29,51225.0,51855.0,Market,False,NaN,NaN
3,08/25/23,06:44:34:905819,06:44:35:829129,1692971075753,NRG,ET,Buy,3.0,NaN,NaN,...,37.26,37.28,37.26,37.28,51857.0,51857.0,Market,False,43974,NaN
4,08/25/23,06:44:36:761912,06:44:37:719205,1692971077639,NRG,E2,Buy,3.0,NaN,NaN,...,36.90,37.69,36.90,37.69,46.0,46.0,Market,False,4516,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,08/25/23,07:34:25:077046,07:34:26:145261,07:34:26,CDNA,ET,Buy,11.0,8.5975,94.5725,...,8.59,8.60,8.59,8.60,48055.0,48068.0,Market,False,44014,NaN
1288,08/25/23,09:43:07:934369,09:43:08:655465,09:43:08,MICS,ET,Buy,77.0,1.3279,102.2483,...,1.29,1.36,1.29,1.36,2520.0,2520.0,Market,False,44104,NaN
1289,08/25/23,09:43:07:934369,09:43:08:655465,09:43:08,MICS,ET,Buy,1.0,1.3250,1.3250,...,1.29,1.36,1.29,1.36,2520.0,2520.0,Market,False,44104,NaN
1290,08/25/23,10:39:06:484051,10:39:07:199139,10:39:07,INZY,ET,Sell,10.0,4.9350,49.3500,...,4.93,4.94,4.93,4.94,175891.0,175891.0,Market,False,44148,NaN


In [191]:
all_trades = pd.read_html("/Users/sanathnair/Developer/trading/data/etrade/all_etrade1_data.html")

[  Transaction Status    Order Description               Date & Time  Quantity  \
 0       Order Placed  Sell 1 CVCO Mkt Day  09/01/23 01:02:52 PM EDT         1   
 1           Executed  Sell 1 CVCO Mkt Day  09/01/23 01:02:52 PM EDT         1   
 
   Price Executed  Commission/Fee  
 0            â          0.0024  
 1         284.56          0.0101  ,
   Transaction Status    Order Description               Date & Time  Quantity  \
 0       Order Placed  Sell 5 AMTB Mkt Day  09/01/23 01:02:20 PM EDT         5   
 1           Executed  Sell 5 AMTB Mkt Day  09/01/23 01:02:20 PM EDT         5   
 
   Price Executed  Commission/Fee  
 0            â          0.0015  
 1        19.4208          0.0107  ,
   Transaction Status     Order Description               Date & Time  \
 0       Order Placed  Sell 10 HONE Mkt Day  09/01/23 01:01:47 PM EDT   
 1           Executed  Sell 10 HONE Mkt Day  09/01/23 01:01:47 PM EDT   
 
    Quantity Price Executed  Commission/Fee  
 0        10       

In [263]:
all_trades_df = pd.DataFrame()
for trade in all_trades:
    all_trades_df = pd.concat([all_trades_df, trade])

In [264]:
all_trades_df

,Transaction Status,Order Description,Date & Time,Quantity,Price Executed,Commission/Fee
0,Order Placed,Sell 1 CVCO Mkt Day,09/01/23 01:02:52 PM EDT,1,â,0.0024
1,Executed,Sell 1 CVCO Mkt Day,09/01/23 01:02:52 PM EDT,1,284.56,0.0101
0,Order Placed,Sell 5 AMTB Mkt Day,09/01/23 01:02:20 PM EDT,5,â,0.0015
1,Executed,Sell 5 AMTB Mkt Day,09/01/23 01:02:20 PM EDT,5,19.4208,0.0107
0,Order Placed,Sell 10 HONE Mkt Day,09/01/23 01:01:47 PM EDT,10,â,0.0022
...,...,...,...,...,...,...
1,Executed,Buy 20 NOTV Mkt Day,06/26/23 09:58:12 AM EDT,20,5.1166,0.0000
0,Order Placed,Buy 1 ODFL Mkt Day,06/26/23 09:58:09 AM EDT,1,â,0.0000
1,Executed,Buy 1 ODFL Mkt Day,06/26/23 09:58:09 AM EDT,1,332.1833,0.0000
0,Order Placed,Buy 1 WH Mkt Day,06/26/23 09:58:05 AM EDT,1,â,0.0000


In [260]:
def parse_etrade_report(df):
    df["Date & Time"] = pd.to_datetime(df["Date & Time"], format="%m/%d/%y %I:%M:%S %p EDT")
    df["Date & Time"] = df["Date & Time"] - pd.Timedelta(hours = 3)

    df[["Action", "Symbol"]] = df["Order Description"].str.split(expand = True)[[0, 2]]

    df = df.drop(["Order Description", "Commission/Fee", "Transaction Status"], axis = 1)
    df = df.drop([0])

    df["Price Executed"] = pd.to_numeric(df["Price Executed"])
    df["Dollar Amt"] = df["Quantity"] * df["Price Executed"]

    df["Broker Executed"] = df["Date & Time"].dt.strftime('%X')
    df["Date"] = df["Date & Time"].dt.strftime("%m/%d/%y")
    
    df = df.drop(columns = ["Date & Time"])
    df = df.rename(columns = {
        "Quantity": "Size",
        "Price Executed": "Price",
    })
    return df

In [265]:
the_correct_one = parse_etrade_report(all_trades_df)


In [267]:
the_correct_one.to_csv("/Users/sanathnair/Developer/trading/data/etrade/correct_etrade1_data.csv", index = False)

In [200]:
all_trades_df[(all_trades_df["Date"] == "08/25/23") & (all_trades_df["Symbol"] == "CDNA")]

,Size,Price,Action,Symbol,Dollar Amt,Broker Executed,Date
1,12,8.6600,Sell,CDNA,103.9200,01:40:28,08/25/23
1,1,8.5975,Buy,CDNA,8.5975,01:34:26,08/25/23
2,11,8.5975,Buy,CDNA,94.5725,01:34:26,08/25/23


In [227]:

all_trades_df.to_csv("/Users/sanathnair/Developer/trading/data/etrade/all_etrade1_data.csv", index = False)

In [228]:
COLUMN_ORDER = ['Date', 'Program Submitted', 'Program Executed', 'Broker Executed', 'Symbol',
                    'Broker', 'Action', 'Size', 'Price', 'Dollar Amt', 'Pre Quote', 'Post Quote',
                    'Pre Bid', 'Pre Ask', 'Post Bid', 'Post Ask', 'Pre Volume', 'Post Volume',
                    'Order Type', 'Split', 'Order ID', 'Activity ID']

def merge_etrade_report(report_df, etrade_df, et_acc):
    merged = pd.merge(left = report_df, right = etrade_df, on = ["Date", "Symbol", "Action", "Broker"])
    merged["Split"] = True
    merged = merged.drop(columns = ["Size_x", "Price_x", "Dollar Amt_x", "Broker Executed_x"])
    merged = merged.rename(columns = {
        "Size_y": "Size",
        "Price_y": "Price",
        "Dollar Amt_y": "Dollar Amt",
        "Broker Executed_y": "Broker Executed"
    })

    merged = merged.reindex(columns = COLUMN_ORDER)
    et = report_df[report_df["Broker"] == et_acc]
    report_df = report_df.drop(et[et["Order ID"].isin(merged["Order ID"])].index)
    report_df = pd.concat([report_df, merged], ignore_index = True)

    return report_df

In [237]:
report_df = pd.read_csv("/Users/sanathnair/Developer/trading/reports/original/report_07_27.csv")
etrade1_df = pd.read_csv("/Users/sanathnair/Developer/trading/data/etrade/all_etrade1_data.csv")
report_df.shape, report_df[report_df["Broker"] == "ET"].shape, etrade1_df[etrade1_df["Date"] == "07/27/23"].shape

((1238, 22), (208, 22), (212, 8))

In [233]:
report_df

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
0,07/27/23,06:44:06:484597,06:44:06:631929,06:44:06,CSX,TD,Buy,3.0,33.0828,99.2484,...,33.08,33.09,33.08,33.09,540349.0,540352.0,Market,False,11337963088,5.804816e+10
1,07/27/23,06:44:08:940244,06:44:10:104554,1690465450047,CSX,ET,Buy,3.0,33.0694,99.2082,...,33.08,33.09,33.06,33.07,541052.0,541658.0,Market,False,41552,NaN
2,07/27/23,06:44:11:331009,06:44:12:575492,1690465452528,CSX,E2,Buy,3.0,NaN,NaN,...,32.95,33.22,32.95,33.22,2777.0,2777.0,Market,False,1162,NaN
3,07/27/23,06:44:13:498423,06:44:20:899234,NaN,CSX,SB,Buy,3.0,NaN,NaN,...,33.07,33.08,33.07,33.08,542672.0,544466.0,Market,False,NaN,NaN
4,07/27/23,06:44:22:495622,06:44:27:956317,NaN,CSX,FD,Buy,3.0,NaN,NaN,...,33.07,33.08,33.07,33.08,544466.0,544573.0,Market,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,07/27/23,11:22:15:711690,11:22:22:628681,NaN,BV,SB,Sell,13.0,NaN,NaN,...,7.67,7.68,7.67,7.68,142819.0,142832.0,Market,False,NaN,NaN
1234,07/27/23,11:22:23:147350,11:22:23:311486,11:22:23,BV,TD,Sell,13.0,7.6707,99.7191,...,7.67,7.68,7.67,7.68,142832.0,142845.0,Market,False,11343583979,5.807441e+10
1235,07/27/23,11:22:24:161536,11:22:24:883080,1690482144837,BV,E2,Sell,13.0,NaN,NaN,...,7.67,7.68,7.67,7.68,130978.0,130978.0,Market,False,1369,NaN
1236,07/27/23,11:22:25:481725,11:22:26:258923,1690482146195,BV,ET,Sell,13.0,7.6708,99.7204,...,7.67,7.68,7.67,7.68,142858.0,142871.0,Market,False,41759,NaN


In [232]:
merge_etrade_report(report_df, etrade1_df, "ET")

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
0,07/27/23,06:44:06:484597,06:44:06:631929,06:44:06,CSX,TD,Buy,3.0,33.0828,99.2484,...,33.08,33.09,33.08,33.09,540349.0,540352.0,Market,False,11337963088,5.804816e+10
1,07/27/23,06:44:11:331009,06:44:12:575492,1690465452528,CSX,E2,Buy,3.0,NaN,NaN,...,32.95,33.22,32.95,33.22,2777.0,2777.0,Market,False,1162,NaN
2,07/27/23,06:44:13:498423,06:44:20:899234,NaN,CSX,SB,Buy,3.0,NaN,NaN,...,33.07,33.08,33.07,33.08,542672.0,544466.0,Market,False,NaN,NaN
3,07/27/23,06:44:22:495622,06:44:27:956317,NaN,CSX,FD,Buy,3.0,NaN,NaN,...,33.07,33.08,33.07,33.08,544466.0,544573.0,Market,False,NaN,NaN
4,07/27/23,06:44:29:545611,06:44:30:441958,NaN,CSX,RH,Buy,3.0,NaN,NaN,...,33.07,33.08,33.07,33.08,544573.0,544673.0,Limit,False,64c274be-4d10-4983-afc4-ade30d55bf05,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,07/27/23,11:15:22:520251,11:15:23:137098,05:15:23,BV,ET,Buy,13.0,7.6550,99.5150,...,7.65,7.66,7.65,7.66,136482.0,136482.0,Market,True,41755,NaN
1238,07/27/23,11:21:21:878220,11:21:23:027050,05:21:23,DUNE,ET,Sell,10.0,9.9900,99.9000,...,9.99,10.00,9.99,10.00,5234.0,5234.0,Market,True,41756,NaN
1239,07/27/23,11:21:43:883791,11:21:44:565568,05:21:44,OKE,ET,Sell,1.0,67.5350,67.5350,...,67.53,67.54,67.53,67.54,1301148.0,1301148.0,Market,True,41757,NaN
1240,07/27/23,11:22:05:211982,11:22:05:998493,05:22:06,BCC,ET,Sell,1.0,99.6600,99.6600,...,99.49,99.77,99.49,99.77,90569.0,90570.0,Market,True,41758,NaN


In [ ]:
# left at 6:14